In [1]:
import pandas as pd
import xarray as xr
import datetime
import os
import numpy as np
import matplotlib.pyplot as plt
import time


from sklearn.metrics import mean_squared_error, r2_score

In [2]:
from dirs import prepdir
from dirs import outputsdir
input_folder_path = prepdir
meta_data = xr.open_dataset(input_folder_path + 'wrfinput_d02')
masks     = xr.open_dataset(input_folder_path + 'basin_masks_filtered.nc')
sweBC     = xr.open_dataarray(input_folder_path + 'snowmaxBC.nc')

In [3]:
## Read in static fields

lat_wrf    = meta_data.variables["XLAT"][0,:,:]
lon_wrf    = meta_data.variables["XLONG"][0,:,:]
z_wrf      = meta_data.variables["HGT"][0,:,:]
vgtyp_wrf  = meta_data.variables["IVGTYP"][0,:,:] ## Table 2: IGBP-Modified MODIS 20-category Land Use Categories
vegfra_wrf = meta_data.variables["VEGFRA"][0,:,:] ## Average canopy cover

lat_wrf    = xr.DataArray(lat_wrf, dims=["lat2d", "lon2d"])
lon_wrf    = xr.DataArray(lon_wrf, dims=["lat2d", "lon2d"])
z_wrf      = xr.DataArray(z_wrf, dims=["lat2d", "lon2d"])
vgtyp_wrf  = xr.DataArray(vgtyp_wrf, dims=["lat2d", "lon2d"]) 
vegfra_wrf = xr.DataArray(vegfra_wrf, dims=["lat2d", "lon2d"])

## Compute slope and aspect
myslopx, myslopy = np.gradient(z_wrf, 9000)
slope_wrf = np.degrees(np.arctan(np.sqrt(myslopx**2 + myslopy**2)))
aspect_wrf = np.degrees(np.arctan2(-myslopy,myslopx))
## Convert aspect to compass direction (clockwise from north)
aspect_q2 = (aspect_wrf > 90) & (aspect_wrf <= 180) ## [90, 180]
aspect_wrf = 90.0 - aspect_wrf
aspect_wrf[aspect_q2] = 360.0 + aspect_wrf[aspect_q2]


gcms = ['cesm2','mpi-esm1-2-lr','cnrm-esm2-1',
        'ec-earth3-veg','fgoals-g3','ukesm1-0-ll',
        'canesm5','access-cm2','ec-earth3',]


variants = ['r11i1p1f1','r7i1p1f1','r1i1p1f2',
            'r1i1p1f1','r1i1p1f1','r2i1p1f2',
            'r1i1p2f1','r5i1p1f1','r1i1p1f1',]

gcm_variants = [f'{item1}_{item2}_ssp370' for item1, item2 in zip(gcms, variants)]

In [4]:
def update_rect (basin_imin, basin_imax, basin_jmin, basin_jmax):
    """Update the bounding box around the basin such that 
    the dimensions are multiples of 8.
    """
    lat_extent = basin_imax - basin_imin
    lat_pad = int(8*(np.ceil(lat_extent/8) )) - lat_extent

    basin_imin = basin_imin - int(np.floor(lat_pad/2))
    basin_imax = basin_imax + int(np.ceil(lat_pad/2))
    # lat_extent = basin_imax - basin_imin

    lon_extent = basin_jmax - basin_jmin
    lon_pad = int(8*(np.ceil(lon_extent/8) )) - lon_extent

    basin_jmin = basin_jmin - int(np.floor(lon_pad/2))
    basin_jmax = basin_jmax + int(np.ceil(lon_pad/2))
    # lon_extent = basin_jmax - basin_jmin
    
    return basin_imin, basin_imax, basin_jmin, basin_jmax

def get_basin_rect(basin_id, update_shape=True):

    basinmask = masks.basin_masks[basin_id]
    mask = basinmask.data.astype(bool)

    ## Extract a rectangle encompassing the basin to maintain spatial structure
    basin_ii, basin_jj = np.nonzero(mask)
    basin_imin, basin_imax = basin_ii.min(), basin_ii.max()+1
    basin_jmin, basin_jmax = basin_jj.min(), basin_jj.max()+1
    
    if update_shape:
        return update_rect(basin_imin, basin_imax, basin_jmin, basin_jmax)
    
    return basin_imin, basin_imax, basin_jmin, basin_jmax

def get_dataarray_basin (basin_id, dataarray):
    basin_imin, basin_imax, basin_jmin, basin_jmax = get_basin_rect(basin_id)
    return dataarray.sel(lat2d=np.arange(basin_imin, basin_imax), lon2d=np.arange(basin_jmin, basin_jmax))

def get_dataarray_gcm_year (gcm_id, years, dataarray):
    return dataarray.sel(gcm=gcm, time=years)
    # return dataarray[gcm_id, years,:].flatten()

In [5]:
train_length = 20
test_years = np.arange(2001, 2101, 5)
train_years = [range(item-train_length, item) for item in test_years]

Data_1 = ['knn_snotel', 'Longitude', 'Latitude']
Data_2 = Data_1 + ['Elevation', 'Slope', 'Aspect', 'Veg-Type', 'Veg-Frac']
Data_3 = Data_2 + ['Cum-fSCA']
Data_4 = Data_3 + ['Cum-precip', 'Cum-snow', 'Mean-temp', 'PDD-sum']
Data_5 = Data_4 + ['ASO-proxy']

In [6]:
src_folder_linreg = 'Fig4_LR_SynthErr_preds/'
src_folder_rf = 'Fig4_RF_SynthErr_preds/'
src_folder_unet = 'Fig4_Unet_preds/'
models = ['linreg', 'RF', 'Unet']
folders = [src_folder_linreg, src_folder_rf, src_folder_unet]
swe_preds = xr.DataArray(np.full((9, 3, test_years.size, 5, 340, 270), np.nan), dims=['gcm','model', 'test_year', 'pred_combo', 'lat2d', 'lon2d'],
            coords={'gcm':gcm_variants,  'model': models, 'test_year' : test_years,
                    'pred_combo':['Data_1', 'Data_2', 'Data_3', 'Data_4', 'Data_5']})

In [7]:
for i in range(2):
    src_folder = folders[i]
    model = models[i]
    for basin_id in range(masks.basin.size):
    # for basin_id in range(0,1):

    ### Will loop over basin_id

        basinname = masks.basin.values[basin_id]
        basinmask = masks.basin_masks[basin_id]
        mask = basinmask.data.astype(bool)

        for gcm_id in range(9):
    ### Will loop over gcm_id, this will be nested inside basin_id loop
            gcm = gcm_variants[gcm_id]

            for test_id in range(test_years.size):
            ### Will loop over test_id, this will be nested inside gcm_id loop


                for data_id, data_var in enumerate([Data_1, Data_2, Data_3, Data_4, Data_5]):

                    preds = np.load(src_folder + f'{basin_id}_{basinname}_{gcm}_{test_years[test_id]}_{data_id}_{model}.npy')

                    swe_preds[gcm_id, i, test_id, data_id].values[mask] = preds


FileNotFoundError: [Errno 2] No such file or directory: 'Fig4_LR_SynthErr_preds_pillowse2_march5/29_Jordan_cnrm-esm2-1_r1i1p1f2_ssp370_2031_2_linreg.npy'

In [ ]:
## next make the error
## method is different between the unet and lr/rf

In [ ]:

pred_combo_list = ['Data_1', 'Data_2', 'Data_3', 'Data_4', 'Data_5']

err_xr = xr.DataArray(np.zeros((masks.basin.size, 9, test_years.size, 3, 5, 2)), dims=['basin','gcm','test_year','model', 'pred_combo', 'metric'],
            coords={'basin' : masks.basin.values, 'gcm':gcm_variants,  'test_year' : test_years, 'model':models,
                    'pred_combo':pred_combo_list, 'metric':['rmse', 'r2']})


In [ ]:
for basin_id in range(masks.basin.size):

### Will loop over basin_id

    basinname = masks.basin.values[basin_id]
    basinmask = masks.basin_masks[basin_id]
    mask = basinmask.data.astype(bool)
    basin_imin, basin_imax, basin_jmin, basin_jmax = get_basin_rect(basin_id, update_shape=True)

    lon_basin_box = (lon_wrf[basin_imin:basin_imax,basin_jmin:basin_jmax])
    lat_basin_box = (lat_wrf[basin_imin:basin_imax,basin_jmin:basin_jmax])

    lon_basin = lon_wrf.values[mask]
    lat_basin = lat_wrf.values[mask]
    ### Extract the basin from the bounding box
    box_i = [i for i,item in enumerate(zip(lon_basin_box.values.flatten(), lat_basin_box.values.flatten())) if item in zip(lon_basin, lat_basin)]
    
    

    ### Take subset of met and swe data here

    swe_basin             = get_dataarray_basin (basin_id, sweBC)

    for gcm_id in range(0,9):
    ### Will loop over gcm_id, this will be nested inside basin_id loop
        gcm = gcm_variants[gcm_id]
        print (f"Modeling {basin_id}_{basinname}_{gcm}")
        for test_id in range(test_years.size):

        ### Will loop over test_id, this will be nested inside gcm_id loop
            Test_SWE = get_dataarray_gcm_year (gcm_id, test_years[test_id], swe_basin).expand_dims(dim='channels', axis=-1)

            # for data_id, data_var in enumerate([Data_2]):
            for data_id, data_var in enumerate([Data_1, Data_2, Data_3, Data_4, Data_5]):

                y_pred = np.load(src_folder + f'{basin_id}_{basinname}_{gcm}_{test_years[test_id]}_{data_id}_Unet.npy')
                
                err_xr.loc[dict(basin=basinname, gcm=gcm, test_year=test_years[test_id], 
                            model='Unet', pred_combo=pred_combo_list[data_id], metric='rmse')] = mean_squared_error(
                                    Test_SWE[:,:,0].values.flatten()[box_i], y_pred.flatten()[box_i])**0.5
                
                err_xr.loc[dict(basin=basinname, gcm=gcm, test_year=test_years[test_id], 
                            model='Unet', pred_combo=pred_combo_list[data_id], metric='r2')] = r2_score(
                                    Test_SWE[:,:,0].values.flatten()[box_i], y_pred.flatten()[box_i])


In [ ]:
%%time
for model in models[0:2]:
    for basin_id in range(masks.basin.size):

        basinname = masks.basin.values[basin_id]
        basinmask = masks.basin_masks[basin_id]
        mask = basinmask.data.astype(bool)

        for gcm_id in range(9):
    ### Will loop over gcm_id, this will be nested inside basin_id loop
            gcm = gcm_variants[gcm_id]

            for test_id in range(test_years.size):
                test_year = test_years[test_id]
            ### Will loop over test_id, this will be nested inside gcm_id loop

                for data_id, data_var in enumerate([Data_1, Data_2, Data_3, Data_4, Data_5]):
                    # y_true = sweBC[gcm_id, test_id+20].values[mask] # possibly wrong test_id ? try selecting year instead 
                    y_true = sweBC.sel(gcm = gcm, time = test_year).values[mask]
                    y_pred = swe_preds.sel(gcm = gcm, model = model, test_year = test_year, pred_combo = pred_combo_list[data_id]).values[mask]
                    err_xr.loc[dict(basin=basinname, gcm=gcm, test_year=test_years[test_id], 
                                model=model, pred_combo=pred_combo_list[data_id], metric='rmse')] = mean_squared_error(y_true, y_pred)**0.5
                    err_xr.loc[dict(basin=basinname, gcm=gcm, test_year=test_years[test_id], 
                                model=model, pred_combo=pred_combo_list[data_id], metric='r2')] = r2_score(y_true, y_pred)
                    
    
                

In [ ]:
err_xr.to_netcdf(outputsdir + 'errors_allbasins.nc')